In [3]:
import os
import subprocess
import yaml
import concurrent.futures
from typing import Dict, List

type = os.getenv('TYPE', 'comfyui') # 'comfyui' or 'comfyui_vace' or 'kijai' or 'gguf'
mode = os.getenv('MODE', 't2v') # 't2v' or 'i2v' or 'both'
resolution = os.getenv('RESOLUTION', 'low') # 'low', 'high', or 'both'
download_threads = int(os.getenv('DOWNLOAD_THREADS', 2))

In [4]:
def get_url_from_file(file: Dict[str, str]) -> List[str]:
    urls = []
    if 'url' in file:
        urls.append(file['url'])
    elif 'low_res' in file and 'high_res' in file:
        if resolution == 'low':
            urls.append(file['low_res']['url'])
        elif resolution == 'high':
            urls.append(file['high_res']['url'])
        elif resolution == 'both':
            urls.append(file['low_res']['url'])
            urls.append(file['high_res']['url'])
    return urls

def download_file(file: Dict[str, str], models_dir: str) -> None:
    # ディレクトリが存在しない場合は作成
    dir_path = os.path.join(models_dir, file["dir"])
    os.makedirs(dir_path, exist_ok=True)
    
    urls = get_url_from_file(file)
    for url in urls:
        # ファイル名を取得
        filename = os.path.basename(url.split("?")[0])
        
        # HuggingFaceのblobリンクをrawリンクに変換（必要な場合）
        download_url = url
        if "/blob/" in download_url:
            download_url = download_url.replace("/blob/", "/resolve/")
        
        print(f"Downloading {filename} to {file['dir']}...")
        
        # curlコマンドを実行してファイルをダウンロード
        cmd = ["curl", "-L", "-C", "-", "-O", "--output-dir", file["dir"], download_url]
        subprocess.run(cmd)
        
        print(f"Downloaded {filename} to {file['dir']}")

In [ ]:
os.chdir('/workspace')
# YAML設定ファイルを読み込む
with open('download_models.yaml', 'r') as f:
    config = yaml.safe_load(f)

# ファイル辞書を初期化（ディレクトリとURLのペアで重複排除）
files_dict = {}

def add_files_to_dict(file_list):
    for file_entry in file_list:
        dir_name = file_entry['dir']
        if dir_name not in files_dict:
            files_dict[dir_name] = []
        
        # URLを取得してリストに追加
        urls = get_url_from_file(file_entry)
        for url in urls:
            if url not in [existing_url for existing_url in files_dict[dir_name]]:
                files_dict[dir_name].append(url)

# デフォルトでupscaleファイルを追加
add_files_to_dict(config['files']['upscale'])

# typeとmodeに基づいてファイルを追加
if type == 'comfyui':
    add_files_to_dict(config['files']['comfyui_common'])
    if mode == 't2v':
        add_files_to_dict(config['files']['comfyui_t2v'])
    elif mode == 'i2v':
        add_files_to_dict(config['files']['comfyui_i2v'])
    elif mode == 'both':
        add_files_to_dict(config['files']['comfyui_t2v'])
        add_files_to_dict(config['files']['comfyui_i2v'])
elif type == 'comfyui_vace':
    add_files_to_dict(config['files']['comfyui_common'])
    if mode == 't2v':
        add_files_to_dict(config['files']['comfyui_vace_t2v'])
    elif mode == 'i2v':
        add_files_to_dict(config['files']['comfyui_vace_i2v'])
    elif mode == 'both':
        add_files_to_dict(config['files']['comfyui_vace_t2v'])
        add_files_to_dict(config['files']['comfyui_vace_i2v'])
elif type == 'kijai':
    add_files_to_dict(config['files']['kijai_common'])
    if mode == 't2v':
        add_files_to_dict(config['files']['kijai_t2v'])
    elif mode == 'i2v':
        add_files_to_dict(config['files']['kijai_i2v'])
    elif mode == 'both':
        add_files_to_dict(config['files']['kijai_t2v'])
        add_files_to_dict(config['files']['kijai_i2v'])
elif type == 'ccuf':
    add_files_to_dict(config['files']['ccuf_common'])
    if mode == 't2v':
        add_files_to_dict(config['files']['ccuf_t2v'])
    elif mode == 'i2v':
        add_files_to_dict(config['files']['ccuf_i2v'])
    elif mode == 'both':
        add_files_to_dict(config['files']['ccuf_t2v'])
        add_files_to_dict(config['files']['ccuf_i2v'])

# /workspace/ComfyUI/models に移動
models_dir = "/workspace/ComfyUI/models"
os.makedirs(models_dir, exist_ok=True)
os.chdir(models_dir)

# 並列ダウンロードを実行
def download_url_to_dir(dir_name, url):
    # ディレクトリが存在しない場合は作成
    dir_path = os.path.join(models_dir, dir_name)
    os.makedirs(dir_path, exist_ok=True)
    
    # ファイル名を取得
    filename = os.path.basename(url.split("?")[0])
    
    # HuggingFaceのblobリンクをrawリンクに変換（必要な場合）
    download_url = url
    if "/blob/" in download_url:
        download_url = download_url.replace("/blob/", "/resolve/")
    
    print(f"Downloading {filename} to {dir_name}...")
    
    # curlコマンドを実行してファイルをダウンロード
    cmd = ["curl", "-L", "-C", "-", "-O", "--output-dir", dir_name, download_url]
    subprocess.run(cmd)
    
    print(f"Downloaded {filename} to {dir_name}")

# すべてのファイルを並列ダウンロード
download_tasks = []
for dir_name, urls in files_dict.items():
    for url in urls:
        download_tasks.append((dir_name, url))

with concurrent.futures.ThreadPoolExecutor(max_workers=download_threads) as executor:
    # 各ファイルのダウンロードをスケジュール
    futures = [executor.submit(download_url_to_dir, dir_name, url) for dir_name, url in download_tasks]
    # すべてのダウンロードが完了するまで待機
    concurrent.futures.wait(futures)

print("All files downloaded successfully!")